In [1]:
import os
import pickle
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

In [2]:
# Paths to data and labels
TRAIN_DATA_PATH = r'D:\UniGe\2\CV\Babel Project\Normalized Data\train_ntu_sk_60_pre.npy'
VAL_DATA_PATH   = r'D:\UniGe\2\CV\Babel Project\Normalized Data\val_ntu_sk_60_pre.npy'
TEST_DATA_PATH  = r'D:\UniGe\2\CV\Babel Project\Normalized Data\test_ntu_sk_60_pre.npy'

TRAIN_LABEL_PATH = r'D:\UniGe\2\CV\Babel Project\BABEL\action_recognition\data\release\train_label_60.pkl'
VAL_LABEL_PATH   = r'D:\UniGe\2\CV\Babel Project\BABEL\action_recognition\data\release\val_label_60.pkl'
TEST_LABEL_PATH  = r'D:\UniGe\2\CV\Babel Project\BABEL\action_recognition\data\release\test_label_60.pkl'

# Hyperparameters
BATCH_SIZE = 64
LR = 1e-3
EPOCHS = 50
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda')

In [4]:
# Custom Dataset
class PoseDataset(Dataset):
    def __init__(self, data_path, label_path):
        self.data = np.load(data_path)  # (N,3,T,25,1)
        with open(label_path, 'rb') as f:
            labels = pickle.load(f)[1][0]
        self.labels = np.array(labels)
        assert len(self.data) == len(self.labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        x = self.data[idx]                         # (3,T,25,1)
        x = np.squeeze(x, axis=-1)                 # (3,T,25)
        y = self.labels[idx]
        return torch.from_numpy(x).float(), y

# LSTM Model
class PoseLSTM(nn.Module):
    def __init__(self, in_dim=75, hidden_dim=128, num_layers=2, num_classes=60, dropout=0.3):
        super().__init__()
        self.lstm = nn.LSTM(in_dim, hidden_dim, num_layers,
                            batch_first=True, dropout=dropout, bidirectional=True)
        self.fc   = nn.Linear(hidden_dim * 2, num_classes)

    def forward(self, x):
        # x: (N,3,T,25)
        N, C, T, V = x.size()
        x = x.permute(0, 2, 3, 1).contiguous()      # (N,T,V,C)
        x = x.view(N, T, V * C)                    # (N,T,75)
        out, (h_n, _) = self.lstm(x)
        # h_n: (num_layers*2, N, hidden_dim)
        h = torch.cat([h_n[-2], h_n[-1]], dim=1)    # (N, hidden_dim*2)
        return self.fc(h)

# Utility: compute accuracy
def compute_accuracy(preds, labels):
    return (preds.argmax(dim=1) == labels).float().mean().item()


In [5]:
# Prepare DataLoaders
dataset_train = PoseDataset(TRAIN_DATA_PATH, TRAIN_LABEL_PATH)
dataset_val   = PoseDataset(VAL_DATA_PATH, VAL_LABEL_PATH)
# dataset_test  = PoseDataset(TEST_DATA_PATH, TEST_LABEL_PATH)

dl_train = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
dl_val   = DataLoader(dataset_val,   batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
# dl_test  = DataLoader(dataset_test,  batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

# Initialize model, loss, optimizer
model = PoseLSTM().to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

# Training loop with history
history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}



In [6]:
def compute_topk_accuracy(preds, labels, k=5):
    # preds: (N, num_classes)
    # labels: (N,)
    topk = preds.topk(k, dim=1).indices  # (N, k)
    labels = labels.view(-1, 1).expand_as(topk)  # (N, k)
    correct = (topk == labels).float().sum(dim=1)  # (N,)
    return correct.mean().item()

def compute_per_class_accuracy(preds, labels, num_classes=60):
    preds = preds.argmax(dim=1)
    correct = preds.eq(labels)
    per_class = []
    for c in range(num_classes):
        mask = labels == c
        total = mask.sum().item()
        if total > 0:
            acc = correct[mask].float().sum().item() / total
            per_class.append(acc)
    return np.mean(per_class)

In [8]:
for epoch in range(1, EPOCHS + 1):
    # Training
    model.train()
    running_loss = 0.0
    running_acc  = 0.0
    for x_batch, y_batch in tqdm(dl_train, desc=f"Epoch {epoch}/{EPOCHS} [Train]"):
        x_batch = x_batch.to(DEVICE)
        y_batch = y_batch.to(DEVICE).long()
        optimizer.zero_grad()
        outputs = model(x_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * x_batch.size(0)
        running_acc  += compute_accuracy(outputs, y_batch) * x_batch.size(0)

    epoch_loss = running_loss / len(dataset_train)
    epoch_acc  = running_acc  / len(dataset_train)
    history['train_loss'].append(epoch_loss)
    history['train_acc'].append(epoch_acc)

    # Validation
    model.eval()
    val_loss = 0.0
    val_acc  = 0.0
    val_top5 = 0.0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for x_batch, y_batch in tqdm(dl_val, desc=f"Epoch {epoch}/{EPOCHS} [Val]"):
            x_batch = x_batch.to(DEVICE)
            y_batch = y_batch.to(DEVICE).long()
            outputs = model(x_batch)
            loss = criterion(outputs, y_batch)
            val_loss += loss.item() * x_batch.size(0)
            val_acc  += compute_accuracy(outputs, y_batch) * x_batch.size(0)
            val_top5 += compute_topk_accuracy(outputs, y_batch, k=5) * x_batch.size(0)
            all_preds.append(outputs.cpu())
            all_labels.append(y_batch.cpu())
    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)
    per_class_acc = compute_per_class_accuracy(all_preds, all_labels, num_classes=60)

    history['val_loss'].append(val_loss / len(dataset_val))
    history['val_acc'].append(val_acc   / len(dataset_val))
    history.setdefault('val_top5', []).append(val_top5 / len(dataset_val))
    history.setdefault('val_per_class_acc', []).append(per_class_acc)

    print(f"Epoch {epoch}: Train loss {epoch_loss:.4f}, acc {epoch_acc:.4f} | "
        f"Val loss {history['val_loss'][-1]:.4f}, acc {history['val_acc'][-1]:.4f}, "
        f"top5 {history['val_top5'][-1]:.4f}, per_class {history['val_per_class_acc'][-1]:.4f}")

Epoch 1/50 [Val]: 100%|██████████| 267/267 [00:01<00:00, 135.56it/s]


Epoch 1: Train loss 2.4851, acc 0.3667 | Val loss 2.5622, acc 0.3542, top5 0.6359, per_class 0.1543


Epoch 2/50 [Val]: 100%|██████████| 267/267 [00:01<00:00, 141.36it/s]


Epoch 2: Train loss 2.4201, acc 0.3776 | Val loss 2.4772, acc 0.3616, top5 0.6595, per_class 0.1706


Epoch 3/50 [Val]: 100%|██████████| 267/267 [00:01<00:00, 140.00it/s]


...

Epoch 48: Train loss 1.1378, acc 0.5721 | Val loss 2.8156, acc 0.3866, top5 0.6957, per_class 0.2427


Epoch 49/50 [Val]: 100%|██████████| 267/267 [00:02<00:00, 131.76it/s]


Epoch 49: Train loss 1.1292, acc 0.5739 | Val loss 2.8274, acc 0.3857, top5 0.6879, per_class 0.2417


Epoch 50/50 [Val]: 100%|██████████| 267/267 [00:01<00:00, 136.54it/s]

Epoch 50: Train loss 1.1177, acc 0.5746 | Val loss 2.8574, acc 0.3801, top5 0.6872, per_class 0.2392


In [9]:
os.makedirs('checkpoints', exist_ok=True)
torch.save(model.state_dict(), 'checkpoints/pose_lstm.pth')
with open('checkpoints/history.pkl', 'wb') as f:
    pickle.dump(history, f)

In [10]:
print("Final Evaluation Metrics (LSTM):")
print(f"Top-1 Accuracy: {history['val_acc'][-1]*100:.3f}%")
print(f"Top-5 Accuracy: {history['val_top5'][-1]*100:.3f}%")
print(f"Per-class Accuracy: {history['val_per_class_acc'][-1]*100:.3f}%")
print(f"Mean Loss: {history['val_loss'][-1]:.4f}")

Final Evaluation Metrics (LSTM):
Top-1 Accuracy: 38.009%
Top-5 Accuracy: 68.717%
Per-class Accuracy: 23.922%
Mean Loss: 2.8574
